In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://bitbucket.org/habedi/datasets/raw/b6769c4664e7ff68b001e2f43bc517888cbe3642/spark/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!rm -rf spark-3.0.2-bin-hadoop2.7.tgz*
!pip -q install findspark pyspark graphframes

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

def init_spark(app_name="HelloWorldApp", execution_mode="local[*]"):
  spark = SparkSession.builder.master(execution_mode).appName(app_name).getOrCreate()
  sc = spark.sparkContext
  return spark, sc

In [ ]:
def main1():
  _, sc = init_spark()
  nums = sc.parallelize([1, 2, 3, 4])
  print(nums.map(lambda x: x*x).collect())

if __name__ == '__main__':
  main1()

[1, 4, 9, 16]


In [247]:

import pyspark.sql.functions as F 

df = spark.read.text("Log")
split_col = f.split(df['value'], ' ')
df = df.withColumn('address', split_col.getItem(0))
df2 = df.drop("value")
df2.show()
hostcount =  df2.groupBy("address").count().select('address' , F.col('count').alias('F'))
hs = hostcount.orderBy('F' , ascending=False)
print(hostcount.count())




+--------------------+
|             address|
+--------------------+
|        199.72.81.55|
|unicomp6.unicomp.net|
|      199.120.110.21|
|  burger.letters.com|
|      199.120.110.21|
|  burger.letters.com|
|  burger.letters.com|
|     205.212.115.106|
|         d104.aa.net|
|      129.94.144.152|
|unicomp6.unicomp.net|
|unicomp6.unicomp.net|
|unicomp6.unicomp.net|
|         d104.aa.net|
|         d104.aa.net|
|         d104.aa.net|
|      129.94.144.152|
|      199.120.110.21|
|ppptky391.asahi-n...|
|  net-1-141.eden.com|
+--------------------+
only showing top 20 rows

+--------------------+-----+
|             address|    F|
+--------------------+-----+
|piweba3y.prodigy.com|17609|
|piweba4y.prodigy.com|11676|
|piweba1y.prodigy.com| 9879|
|  alyssa.prodigy.com| 7880|
| siltb10.orl.mmc.com| 7573|
|piweba2y.prodigy.com| 5936|
|  edams.ksc.nasa.gov| 5458|
|        163.206.89.4| 4906|
|         news.ti.com| 4881|
|disarray.demon.co.uk| 4486|
|   poppy.hensa.ac.uk| 4327|
|www-d1.proxy.ao

In [236]:
import pyspark.sql.functions as F 
from pyspark.sql.functions import count, avg


df = df.withColumn('time', split_col.getItem(3))
df2 = df.drop("value")
# df2.show()
spli = f.split(df2['time'], ':')
df2 = df.withColumn('day', spli.getItem(0))

df3 = df2.drop("value")
df3 = df3.drop("time")
hostdayinoutcount =  df3.groupBy("address", "day").count().select('address' , 'day' , F.col('count').alias('Frequency'))
hostdayinoutcount.orderBy('address' , ascending=True).show()

# df3.show()

+--------------------+------------+---------+
|             address|         day|Frequency|
+--------------------+------------+---------+
|         ***.novo.dk|[11/Jul/1995|       16|
|      007.thegap.com|[06/Jul/1995|       30|
|      007.thegap.com|[09/Jul/1995|       11|
|      007.thegap.com|[23/Jul/1995|        4|
|01-dynamic-c.rott...|[28/Jul/1995|        1|
|01-dynamic-c.woki...|[05/Jul/1995|        3|
|01-dynamic-c.woki...|[27/Jul/1995|        9|
|01-dynamic-c.woki...|[10/Jul/1995|       11|
|01-dynamic-c.woki...|[12/Jul/1995|        5|
|02-dynamic-c.woki...|[27/Jul/1995|        4|
|02-dynamic-c.woki...|[07/Jul/1995|        9|
|03-dynamic-c.woki...|[04/Jul/1995|       12|
|03-dynamic-c.woki...|[20/Jul/1995|        3|
|04-dynamic-c.rott...|[03/Jul/1995|       22|
|04-dynamic-c.woki...|[04/Jul/1995|       28|
|05-dynamic-c.rott...|[06/Jul/1995|       12|
|05-dynamic-c.rott...|[05/Jul/1995|       14|
|05-dynamic-c.woki...|[15/Jul/1995|        9|
|06-dynamic-c.rott...|[14/Jul/1995

In [245]:
# df.show()
df = df.drop("time")
df = df.drop("address")
# df.show()
df2 = df.filter(df.value.like('%.gif%'))
print(df2.count())

1034638


In [257]:
hs = hostcount.orderBy('F' , ascending=True)
df = hs.filter("F > 3 and")
expr = "^((?!-)[A-Za-z0-9-]{1,63}(?<!-)\\.)+[A-Za-z]{2,6}$"
dk = df.filter(df["address"].rlike(expr))

dk.show()

+--------------------+---+
|             address|  F|
+--------------------+---+
|hwwmac.larc.nasa.gov|  4|
| braemar.demon.co.uk|  4|
|berlin.vas.viewlo...|  4|
|dr4umac1.med.virg...|  4|
| scorch.doc.ic.ac.uk|  4|
|blv-pm1-ip9.halcy...|  4|
| dram.cmu.susx.ac.uk|  4|
|mac12.bnf23.ulava...|  4|
|  guest7.cni.mid.net|  4|
|   bend6.bendnet.com|  4|
|ix-sj12-17.ix.net...|  4|
|n113.solano.commu...|  4|
|ix-lv4-04.ix.netc...|  4|
|    dwkm107.usa1.com|  4|
|      lom000.wwa.com|  4|
|annex-v32bis-47.s...|  4|
|cust46.max1.ffx1....|  4|
|ix-phi1-01.ix.net...|  4|
|   mufasa.ee.pdx.edu|  4|
|ppp1-06.inre.asu.edu|  4|
+--------------------+---+
only showing top 20 rows



In [278]:
df = spark.read.text("Log")
split_col = f.split(df['value'], '"')
df = df.withColumn('statusbebad', split_col.getItem(2))
df2 = df.drop("value")
split_col2 = f.split(df['statusbebad'], ' ')
df2 = df2.withColumn('status code', split_col2.getItem(1))
df2 = df2.drop("statusbebad")

# df2.show()

# df2.show()
statuscount =  df2.groupBy("status code").count().select('status code'  , F.col('count').alias('Frequency'))
statuscount.orderBy('Frequency' , ascending=False).show(5)


+-----------+---------+
|status code|Frequency|
+-----------+---------+
|        200|  1709982|
|        304|   133349|
|        302|    46925|
|        404|    10878|
|        500|       62|
+-----------+---------+
only showing top 5 rows

